In [5]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
from typing import List
import numpy as np
import notebook_helper
import mprompt.viz
import openai
from pprint import pprint
import joblib
from collections import defaultdict
from mprompt.config import RESULTS_DIR
import json
openai.api_key_path = os.path.expanduser('~/.OPENAI_KEY')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def get_rows_voxels():
    '''Select rows from fitted voxels
    '''
    r = (pd.read_pickle('../results/results_fmri.pkl')
        .sort_values(by=['top_score_synthetic'], ascending=False))
    r['id'] = "('" + r['top_explanation_init_strs'].str.replace(' ', '_').str.slice(stop=20) + "', '" + r['subject'] + "', " + r['module_num'].astype(str) + ")"

    # manually pick some voxels
    # with pd.option_context('display.max_rows', None, 'display.max_colwidth', 200):
    #     display(r.sort_values(by=['top_score_synthetic'], ascending=False)[
    #         ['top_explanation_init_strs', 'subject', 'module_num', 'top_score_synthetic', 'frac_top_ngrams_module_correct', 'id', 'top_ngrams_module_correct']
    #     ].round(3).reset_index(drop=True).head(50))


    # expls = ['baseball','animals','water','movement','religion','time','technology']
    # interesting_expls = ['food', 'numbers', 'physical contact', 'time', 'laughter', 'age', 'clothing']
    # voxels = [('movement', 'UTS01',	7), ('numbers', 'UTS03', 55), ('time', 'UTS03', 19), ('relationships', 'UTS01', 21),
            #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
    # voxels = [('numbers', 'UTS03', 55), ('time', 'UTS03', 19),
            #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
    # voxels = [('movement', 'UTS01',	7),('relationships', 'UTS01', 21) ('passing of time	UTS02	4)]
    # voxels = [('relationships', 'UTS02', 9), ('time', 'UTS02', 4), ('looking or staring', 'UTS03', 57), ('food and drinks', 'UTS01', 52), ('hands and arms', 'UTS01', 46)]

    # mar 21 - voxels spread across categories
    # voxels = [
    #     # belong to previous categories
    #     ('hands and arms', 'UTS01', 46),
    #     ('measurements and numbers', 'UTS02', 48),
    #     ('locations', 'UTS03', 87),
    #     ('time', 'UTS02', 4),
    #     ('physical injury or discomfort', 'UTS01', 35),
    #     ('feelings and emotions', 'UTS02', 104),
    #     ('relationships', 'UTS02', 9),

    #     # new voxels
    #     ('food and drinks', 'UTS01', 52),
    #     ('sound', 'UTS02', 81),
    #     ('hands and arms', 'UTS01', 46),
    # ]

    # mar 22 - UTS02 voxels
    voxels_dict = json.load(open(f'voxel_select/uts02_concepts_pilot_mar22.json', 'r'))
    d = defaultdict(list)
    for k in voxels_dict.keys():
        d['voxels'].extend([tuple(vox) for vox in voxels_dict[k]])
        d['category'].extend([k] * len(voxels_dict[k]))
    d = pd.DataFrame(d)
    voxels = d.voxels.values.tolist()

    # put all voxel data into rows DataFrame
    rows = []
    expls = []
    for vox in voxels:
        expl, subj, vox_num = vox
        try:
            rows.append(r[(r.subject == subj) & (r.module_num == vox_num)].iloc[0])
            expls.append(expl)
        except:
            print('skipping', vox)
    rows = pd.DataFrame(rows)
    rows['expl'] = expls

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 200):
        # display(rows[['subject', 'module_num', 'expl', 'top_explanation_init_strs', 'top_ngrams_module_correct']])

    return rows

def get_rows_huth():
    '''Select rows corresponding to 2016 categories
    '''
    huth2016_categories = json.load(open('huth2016clusters.json', 'r'))
    r = pd.DataFrame.from_dict({'expl': huth2016_categories.keys(), 'top_ngrams_module_correct': huth2016_categories.values()})
    return r

# Generate story

In [11]:
# version = 'v4'
# EXPT_NAME = 'huth2016clusters_mar21_i_time_traveled'
# rows = get_rows_huth()

# EXPT_NAME = 'relationships_mar9'
EXPT_NAME, version = ('voxels_mar21_hands_arms_emergency', 'v4_noun')
rows = get_rows_voxels()

expls = rows.expl.values
examples_list = rows.top_ngrams_module_correct
prompts = notebook_helper.get_prompts(expls, examples_list, version, n_examples=4)
for p in prompts:
    print(p)
PV = notebook_helper.get_prompt_templates(version)

Write the beginning paragraph of an interesting story told in first person. The story should place be about "hands and arms". Make sure it contains several words related to "hands and arms", such as "grab his arms", "put his arms", "put my arms", "grabbed their hands".
Write the next paragraph of the story, but now make it about "measurements and numbers". Make sure it contains several words related to "measurements and numbers", such as "fifty pound animals", "am turning forty", "sixty fifth birthday", "two inch yellow".
Write the next paragraph of the story, but now make it about "locations". Make sure it contains several words related to "locations", such as "in an alcove", "a park bench", "onto the subway", "on the subway".
Write the next paragraph of the story, but now make it about "time". Make sure it contains several words related to "time", such as "weeks became months", "weekends became weeks", "later she revised", "five years four".
Write the next paragraph of the story, but

In [12]:
paragraphs = mprompt.llm.get_paragraphs(prompts, prefix_first=PV['prefix_first'], prefix_next=PV['prefix_next'])
rows['prompt'] = prompts
rows['paragraph'] = paragraphs
for para in paragraphs:
    print(para)
    # pprint(para)

cached!
cached!
cached!
cached!
cached!
cached!
cached!
cached!
cached!
cached!
I never realized how much I relied on my hands and arms until the accident. It happened so fast, I didn't even have time to react. One moment, I was walking down the street, lost in thought, and the next thing I knew, a car came careening around the corner and hit me. I remember feeling the impact and then being thrown to the ground. When I tried to get up, I couldn't move. It was like my body had shut down, and all I could do was lay there while people rushed around me, grabbing my arms and trying to help me.
As I lay there, waiting for the ambulance to arrive, my mind started to wander. I couldn't feel anything below my waist, so I started to think about all the things I had taken for granted. Like being able to walk, run, and even just stand up on my own. It was then that I realized how much our lives are defined by measurements and numbers. From the weight of the fifty pound animals I used to care for a

In [13]:
STORIES_DIR = join(RESULTS_DIR, 'stories')
os.makedirs(join(STORIES_DIR, EXPT_NAME), exist_ok=True)
joblib.dump(rows, join(STORIES_DIR, EXPT_NAME, 'rows.pkl'))

['/home/chansingh/mprompt/results/stories/voxels_mar21_hands_arms_emergency/rows.pkl']